In [1]:
import pandas as pd
import numpy as np

data_path = "./data/House_Price.csv"
df = pd.read_csv(data_path, header=0)
# removing columns with NaN values (waterbody and n_hos_beds)
df.dropna(axis=1, inplace=True)
# removing bus_ter column (it has the same value for each row)
df.drop(["bus_ter"], axis=1, inplace=True)
# encoding airport values
airport_mapping = {'YES': 1, 'NO': 0}
df['airport'] = df['airport'].map(airport_mapping)
df.head()

,price,crime_rate,resid_area,air_qual,room_num,age,dist1,dist2,dist3,dist4,teachers,poor_prop,airport,n_hot_rooms,rainfall,parks
0,24.0,0.00632,32.31,0.538,6.575,65.2,4.35,3.81,4.18,4.01,24.7,4.98,1,11.1920,23,0.049347
1,21.6,0.02731,37.07,0.469,6.421,78.9,4.99,4.70,5.12,5.06,22.2,9.14,0,12.1728,42,0.046146
2,34.7,0.02729,37.07,0.469,7.185,61.1,5.03,4.86,5.01,4.97,22.2,4.03,0,101.1200,38,0.045764
3,33.4,0.03237,32.18,0.458,6.998,45.8,6.21,5.93,6.16,5.96,21.3,2.94,1,11.2672,45,0.047151
4,36.2,0.06905,32.18,0.458,7.147,54.2,6.16,5.86,6.37,5.86,21.3,5.33,0,11.2896,55,0.039474


In [2]:
# splitting data into train set and test set
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state=0)

In [3]:
# regression tree implementations
import sys

class Node:
    def __init__(self, atribute=None, threshold=None, value=None, left=None, right=None):
        self.atribute = atribute # which atribute are we splitting on
        self.threshold = threshold  # threshold value for the split
        self.value = value  # value to predict at this node (for leaf nodes)
        self.left = left  # left subtree
        self.right = right  # right subtree

def compute_RSS(y_col: pd.DataFrame, y_pred: float):
    rss = 0
    for y in y_col:
        rss += (y-y_pred)**2
    return rss

def find_best_atr_and_threshold(data: np.array, atributes: list, target_name: str):
    # this assumes the order of the atributes is the same as the order of data columns
    # this also assumes the target is the last atribute
    min_RSS = sys.float_info.max
    best_atribute = None
    best_threshold = -1.0
    
    for atr_index, atribute in enumerate(atributes):
        min_RSS_for_atribute = sys.float_info.max
        best_threshold_for_atribute = -1.0
        # Sorting the data by the curent column
        sorted_indices = np.argsort(data[:, atr_index])
        data = data[sorted_indices]

        for i in range(data.shape[0]-1): # TODO if column has only 0s and 1s this is a waste of time (but im too lazy to fix it rn)
            # splitting the data into two and calculating RSS for each
            data1 = data[:i+1, :]
            data2 = data[i+1:, :]
            cur_RSS = compute_RSS(data1[:, -1], data1[:, -1].mean()) + compute_RSS(data2[:, -1], data2[:, -1].mean())
            if cur_RSS < min_RSS_for_atribute:
                min_RSS_for_atribute = cur_RSS
                best_threshold_for_atribute = data[i, atr_index]
        if min_RSS_for_atribute < min_RSS:
            min_RSS = min_RSS_for_atribute
            best_threshold = best_threshold_for_atribute
            best_atribute = atribute
    return best_atribute, best_threshold

def fit_regression_tree(data: pd.DataFrame, target_name: str, min_instances = 39, max_depth=10):
    y = data[target_name]
    X = data.drop(target_name, axis=1)
    data = pd.concat([X, y], axis=1) # putting y in the last column of data
    data = np.array(data) # converting data to np array
    atributes = X.columns.tolist() # getting the list of atributes
    return fit_regression_tree_recursion(data, atributes, target_name, min_instances, max_depth) # returns the root node of the regression tree

def fit_regression_tree_recursion(data: np.array, atributes:list, target_name:str, min_instances, max_depth, cur_depth=0): # builds the regression tree via recursion
    if cur_depth > max_depth:
        # create leaf node and return it
        avg_y = np.mean(data[:, -1]) # calculate the mean value of all y 
        return Node(None, None, value=avg_y, left=None, right=None)
    if data.shape[0] <= min_instances:
        # create leaf node and return it
        avg_y = np.mean(data[:, -1]) # calculate the mean value of all y 
        return Node(None, None, value=avg_y, left=None, right=None)
    # get the best current atribute and best threshold
    best_atr, best_threshold = find_best_atr_and_threshold(data, atributes, target_name)
    # sort the data for best atribute
    sorted_indices = np.argsort(data[:, atributes.index(best_atr)])
    data = data[sorted_indices]
    # split the data on the threshold
    i = np.where(data[:, atributes.index(best_atr)] == best_threshold)[0][-1]
    data_left = data[:i+1, :]
    data_right = data[i+1:, :]
    # recursively build left and right subtree and append them to current node
    node_left = fit_regression_tree_recursion(data_left, atributes, target_name, min_instances, max_depth, cur_depth = cur_depth + 1)
    node_right = fit_regression_tree_recursion(data_right, atributes, target_name, min_instances, max_depth, cur_depth = cur_depth + 1)
    return Node(atribute = best_atr, threshold=best_threshold, value=None, left=node_left, right=node_right)

root = fit_regression_tree(df_train, "price", min_instances=39)

In [4]:
def print_tree(node, depth=0, prefix="Root: "):
    if node is not None:
        print("  " * depth + prefix + f"{node.atribute} {node.threshold} {node.value}")
        if node.left or node.right:
            print_tree(node.left, depth + 1, "L-> ")
            print_tree(node.right, depth + 1, "R-> ")

print_tree(root)

Root: poor_prop 8.1 None
  L-> room_num 7.42 None
    L-> room_num 6.635 None
      L-> dist3 1.52 None
        L-> None None 50.0
        R-> room_num 6.538 None
          L-> poor_prop 7.79 None
            L-> None None 23.869230769230764
            R-> None None 20.46
          R-> None None 27.06153846153847
      R-> resid_area 48.1 None
        L-> dist1 1.55 None
          L-> None None 50.0
          R-> room_num 7.079 None
            L-> None None 29.745454545454546
            R-> None None 34.352380952380955
        R-> None None 41.3
    R-> None None 44.929166666666674
  R-> poor_prop 14.98 None
    L-> room_num 6.593 None
      L-> n_hot_rooms 15.196 None
        L-> teachers 21.3 None
          L-> crime_rate 14.3337 None
            L-> resid_area 39.69 None
              L-> None None 19.493548387096773
              R-> None None 20.734375
            R-> None None 10.9
          R-> parks 0.054831218 None
            L-> None None 22.528205128205123
            R-

In [5]:

for i in df_train["airport"].sort_values():
    print(i)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [6]:
print(np.array(df_train).shape)

(404, 16)


In [7]:
data = np.array(([0,0,1], [3,2,1], [1,2,2], [2,1,2], [4,-1,3]))

sorted_indices = np.argsort(data[:, 1])
data = data[sorted_indices]

print(data)

[[ 4 -1  3]
 [ 0  0  1]
 [ 2  1  2]
 [ 3  2  1]
 [ 1  2  2]]
